<a href="https://colab.research.google.com/github/WittmannF/awari-calculadora-imoveis-may-20/blob/master/1-web-scraping/Atividade_Coleta_e_Manipulac%CC%A7a%CC%83o_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ATIVIDADE: Coleta e Manipulação de Dados de Aluguéis de SP

## SOLUÇÃO: Vídeo com Passo a Passo
- [Parte 1](https://www.loom.com/share/8efbccb6ce1f46e88e10b6a2a336f43b)
- [Parte 2](https://www.loom.com/share/41ce7aa9836148e0972deb352f0e84e0)

## Objetivo
1. Coletar os dados de aluguéis de imóveis dos 6 primeiras páginas de todas as zonas de São Paulo. Utilizar o [arquivo desenvolvido ao longo das últimas aulas](https://colab.research.google.com/drive/1lWX_YvSgfC5A37iE-h9dRfTFmA3LFYrD) como ponto de partida.
2. Gerar uma base de dados .csv semelhante à do [seguinte link](https://github.com/WittmannF/machine-learning-web-app/blob/master/1-scraping/dados_calculadora_imoveis.csv)
3. Criar um repositório para esta atividade e colocar o código e a base de dados criada
4. Enviar o repositório para a gente utilizando o [seguinte formulário](https://airtable.com/shrE7EK8MSFMeR1xh)

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
from time import sleep

In [ ]:
requests.get('https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145').content

b'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<!doctype html>\n<html class="no-js" lang="pt-BR" data-site="MLB"\n      data-country="BR"\n      data-device="desktop">\n\n<head prefix="">\n\t<title>Casas Aluguel em S\xc3\xa3o Paulo Zona Sul no Mercado Livre Brasil\n</title>\n\n\t<link rel="preconnect" href="//analytics.mlstatic.com"/>\n\t<link rel="preconnect" href="//resources.mlstatic.com"/>\n\t<link rel="preconnect" href="//static.mlstatic.com"/>\n\t<link rel="preconnect" href="https://www.google-analytics.com"/>\n\t<link rel="preconnect" href="https://www.google.com"/>\n\t<link rel="preconnect" href="https://data.mercadolibre.com"/>\n\t<link rel="preconnect" href="https://http2.mlstatic.com"/>\n\t\n\t\n\t\t<link rel="preconnect" href="https://www.google.com.br"/>\n\t\n\n\n\t\n\t\t<link rel="preconnect" href="https://stats.g.doubleclick.net"/>\n\t\n\t\n\t\t<link rel="preconnect" href="//analytics.mercadolivre.com.br"/>\n\t\t<link rel="preconnect" href="//analytics.mercadolivre.

In [ ]:
ZONAS = ['norte', 'sul', 'leste', 'oeste']
PAGINAS = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', '_Desde_241']
URL_ML = 'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}'

# Podemos definir dois padrões regex, um para o numero de quartos e outro para a area
re_quarto = '\| (.*) quarto'
re_area = '> (.*) m²'
re_quarto_area_ausente = "> (.*) quarto"

In [ ]:
URL_ML.format('norte', '')

'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/'

In [ ]:
def coletar_dados_ml(url, zona):
  c = requests.get(url).content
  soup = BeautifulSoup(c)
  atributos = soup.find_all('div', {"class": "item__attrs"})
  precos = soup.find_all('span', {"class": "price__fraction"})

  lista_precos = []
  for s in precos:
    p = re.findall('<span class="price__fraction">(.*)</span>', str(s))
    lista_precos.append(int(p[0].replace('.','')))

  # Inicializar duas listas vazias, uma para a area e outra para o numero de quartos
  lista_areas = []
  lista_quartos = []
  lista_zonas = []
  for a in atributos:
    # Extrair area e número de quartos
    n_quartos = re.findall(re_quarto, str(a))
    area = re.findall(re_area, str(a))
    
    # Atribuir np.nan quando o número de quartos estiver ausente 
    if n_quartos==[]:
      n_quartos = re.findall(re_quarto_area_ausente, str(a))

      if n_quartos==[]:
        n_quartos=np.nan
      else:
        n_quartos=n_quartos[0]
  
    else:
        n_quartos=n_quartos[0]

    if area==[]:
      area=np.nan
    else:
      area=area[0]
    #print(n_quartos)

    # Anexar às listas acima
    lista_quartos.append(n_quartos)
    lista_areas.append(area)
    lista_zonas.append(zona)

  dados = {
      'zona': lista_zonas,
      'quartos': lista_quartos,
      'area': lista_areas,
      'preco': lista_precos
      }

  df = pd.DataFrame(dados)
  return df

In [ ]:
lista_dfs = []
for zona in ZONAS:
  for pg in PAGINAS:
    url = URL_ML.format(zona, pg)
    print('Carregando a URL', url)

    df = coletar_dados_ml(url, zona)
    lista_dfs.append(df)
    sleep(1)
    

Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_145
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_193
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_241
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Carregando a URL https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
Carregando a URL https://imoveis.mercadolivre.com

In [ ]:
dados = pd.concat(lista_dfs)

In [ ]:
dados

,zona,quartos,area,preco
0,norte,1,70,600
1,norte,2,70,850
2,norte,1,40,850
3,norte,1,50,950
4,norte,1,160,790
...,...,...,...,...
43,oeste,3,200,15000
44,oeste,2,120,6000
45,oeste,4,350,7900
46,oeste,2,240,4600


In [ ]:
dados['quartos'] = dados['quartos'].astype(float)
dados['area'] = dados['area'].astype(float)

In [ ]:
dados.to_csv('dados_calculadora_imoveis_aula.csv', index=False)

In [ ]:
pd.read_csv("dados_calculadora_imoveis_aula.csv")

,zona,quartos,area,preco
0,norte,1.0,70.0,600
1,norte,2.0,70.0,850
2,norte,1.0,40.0,850
3,norte,1.0,50.0,950
4,norte,1.0,160.0,790
...,...,...,...,...
1147,oeste,3.0,200.0,15000
1148,oeste,2.0,120.0,6000
1149,oeste,4.0,350.0,7900
1150,oeste,2.0,240.0,4600


In [ ]:
ls

dados_calculadora_imoveis_aula.csv  sample_data/
